這次作業我們想分析「帶有 emoji 的text 」的情緒，以及哪些 emoji 本身情緒會不同於它所在的 text 的情緒。

作業分成 3 個部分，分別是前處理、情緒分析contingency table、情緒衝突的emoji詞頻。

**`每一部分皆有需呈現的結果`** ，請務必依指示完成作業。



# 第一部分：前處理

1. 讀取語料 emoji_1000.csv ，並使用 `emoji module` 將每一句話出現的 emoji 轉換(demojize)成它的英文解釋。

2. 讀取英文解釋與繁體中文的對應表 `emoji_defintion.csv` ，並將 **前一步的英文解釋** 轉換成繁體中文。
   
   ⚠️注意：emoji 的英文解釋前後都有冒號，轉換成中文後請用**底線**將 emoji 連接在**句尾**⚠️

   ```
    "🌟天婦羅（時蔬）"
   會先變成： 天婦羅（時蔬）:glowing_star:
   再變成： 天婦羅（時蔬）_發光的星星

    "看的真的是又愛又虐😍"
   會先變成：看的真的是又愛又虐:smiling_face_with_heart-eyes: 
   再變成：看的真的是又愛又虐_心眼笑臉
   ```

3. 請依照 下方回答範例 df的欄位名命名，**顯示出 df[202:207]**，不要顯示 df.head()❗️❗️
  
   ⚠️注意：當一個句子有多個 emoji 時，請以半型逗號連結成string，再將這個string放入emoji_only 欄位。
   
   （請參考下方範例中 id 205 的句子）


> 配分方式：

- step 1.  (10%)
- step 2. (10%)
- step 3. (20%)  df[202:207]

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%pip install emoji --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f00b6ca6b57e4a5d8966621703f98b87d520368ae66f207fa203d9cb2931a776
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [3]:
import pandas as pd
import emoji
import re
from collections import Counter

In [4]:
emoji_df = pd.read_csv('/content/gdrive/MyDrive/NTU GIL/CLLT ta/emoji_sentiment_analysis (hw5)/emoji_1000.csv')
emoji_df.head()

,Unnamed: 0,sentence
0,0,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️
1,1,為今日難吃的涼麵做個紀錄😤
2,2,前幾天的照片真的慘不忍睹😂
3,3,🌟天婦羅（時蔬）
4,4,看的真的是又愛又虐😍


In [6]:
'''把emoji轉成英文（10%）'''

# emoji package: demojize
emo_list = emoji_df['sentence'].to_list()
emo_text = [emoji.demojize(emo) for emo in emo_list ]
emo_text[:3]

['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！:red_heart:',
 '為今日難吃的涼麵做個紀錄:face_with_steam_from_nose:',
 '前幾天的照片真的慘不忍睹:face_with_tears_of_joy:']

In [5]:
''' 將 前一步的英文解釋 轉換成繁體中文 (10%)'''

# Load emoji_definition as map_df
map_df = pd.read_csv('/content/gdrive/MyDrive/NTU GIL/CLLT ta/emoji_sentiment_analysis (hw5)/sub_emoji_df.csv')
map_df.head()

,emoji,en,clean_en,zh-TW,zh
0,🥇,:1st_place_medal:,1st place medal,第一名獎牌,:金牌:
1,🥈,:2nd_place_medal:,2nd place medal,第二名獎牌,:银牌:
2,🥉,:3rd_place_medal:,3rd place medal,第三名獎牌,:铜牌:
3,🅰,:A_button_(blood_type):,A button (blood type),A鈕（血型）,:A型血:
4,♈,:Aries:,Aries,白羊座,:白羊座:


In [7]:
'''1.製作以English為key、中文為value的dict'''
trans_dict = {}
for idx, row in map_df.iterrows():
  clean_en = re.sub(' ', '_', row['clean_en'])
  trans_dict[clean_en] = row['zh-TW']


'''2.emoji_only(zh-TW) list'''
emo_zh = []
for emo in emo_text:
  # regex抓出':'之間的的emoji
  emos = re.findall(r':([^:\s]+):', emo)
  
  # 當一個句子有多個 emoji 時，先個別對照中文翻譯，再以半型逗號連結成string
  if len(emos) > 1: 
    zs = [trans_dict.get(e, 'n') for e in emos]
    zh_str = ', '.join(zs) 
    emo_zh.append(zh_str)
  
  # 句中僅一個emoji，直接查找dict
  else:
    zh = trans_dict.get(emos[0])
    emo_zh.append(zh)


'''3.text_only list'''
txt_only = [re.sub(r':.+:', '', emo) for emo in emo_text]


# 檢視一下
print(emo_list[:5])
print(emo_text[:5])
print(txt_only[:5])
print(emo_zh[:5])

['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️', '為今日難吃的涼麵做個紀錄😤', '前幾天的照片真的慘不忍睹😂', '🌟天婦羅（時蔬）', '看的真的是又愛又虐😍']
['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！:red_heart:', '為今日難吃的涼麵做個紀錄:face_with_steam_from_nose:', '前幾天的照片真的慘不忍睹:face_with_tears_of_joy:', ':glowing_star:天婦羅（時蔬）', '看的真的是又愛又虐:smiling_face_with_heart-eyes:']
['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！', '為今日難吃的涼麵做個紀錄', '前幾天的照片真的慘不忍睹', '天婦羅（時蔬）', '看的真的是又愛又虐']
['紅心', '臉上有蒸汽從鼻子裡冒出來', '滿臉喜悅的淚水', '發光的星星', '心眼笑臉']


In [8]:
dict_1000 = []
for i in range(999):
    t = (txt_only[i], emo_zh[i])
    txt_emo = '_'.join(t)
    dict_1000.append({
        'sentence': emo_list[i],
        'text_only': txt_only[i],
        'emo_only': emo_zh[i],
        'text_emoji': txt_emo,
    })

pre_df = pd.DataFrame(dict_1000)
pre_df[202:207]

,sentence,text_only,emo_only,text_emoji
202,我就突然啜泣哭了😭（我原本以為只會默默流眼淚,我就突然啜泣哭了（我原本以為只會默默流眼淚,大聲哭泣的臉,我就突然啜泣哭了（我原本以為只會默默流眼淚_大聲哭泣的臉
203,每次都說粗重的我來，妳在旁邊等😂,每次都說粗重的我來，妳在旁邊等,滿臉喜悅的淚水,每次都說粗重的我來，妳在旁邊等_滿臉喜悅的淚水
204,不見不散💕,不見不散,兩顆心,不見不散_兩顆心
205,放個假沒想到也換我發生悲劇了😭😭😭,放個假沒想到也換我發生悲劇了,"大聲哭泣的臉, 大聲哭泣的臉, 大聲哭泣的臉","放個假沒想到也換我發生悲劇了_大聲哭泣的臉, 大聲哭泣的臉, 大聲哭泣的臉"
206,希望照片妳滿意啦👊🏻,希望照片妳滿意啦,迎面而來的拳頭淺膚色,希望照片妳滿意啦_迎面而來的拳頭淺膚色


# 第二部分：情緒分析

1. 跑完前處理後，我們會得到一個 df，每一列都是一句 text 的相關資料。

2. 請完成以下 setup 的 block，載入並利用此 classifier，以 「欄位」為單位，分別跑 text_only, emo_only, text_emoji 三組的情緒分析。
3. 成功跑出情緒分析結果後，請與原本的 df 合併，**顯示出一個 senti_df**，請依照下方 `回答範例 DataFrame` 來命名欄位。
   
   - text_only 的情緒分析結果 存在 text_senti 欄
   - emo_only 的情緒分析結果 存在 emoji_senti 欄	
   - text_emoji 的情緒分析結果 存在 txt_emo_senti 欄


3. 利用前一步做好的 df，分別輸出 (text_senti, emoji_senti)、(text_senti, txt_emo_senti)、(emoji_senti, txt_emo_senti) 的 contingency table，**總共3️⃣個** 。請參考下方範例 block。

> 配分方式
- DataFrame (25%)
- contingency table (15%，每一個table 5%)

In [ ]:
## 安裝模型
!pip install transformers datasets

In [10]:
'''''
Model source: https://huggingface.co/liam168/c2-roberta-base-finetuned-dianping-chinese/commit/952591d4ffb6df7b674eba74c4e2bb5dc9cb3128#d2h-611991
'''''

## 載入模型 (請參考連結中的示範code)
from transformers import AutoModelForSequenceClassification , AutoTokenizer, pipeline
model_name = "liam168/c2-roberta-base-finetuned-dianping-chinese"
class_num = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=class_num)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 載入classifier (請參考連結中的示範code)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)


In [11]:
senti_df = pre_df.copy()
senti_df['text_senti'] = [lbl['label'] for lbl in classifier(senti_df['text_only'].tolist())]
senti_df['emoji_senti'] = [lbl['label'] for lbl in classifier(senti_df['emo_only'].tolist())]
senti_df['txt_emo_senti'] = [lbl['label'] for lbl in classifier(senti_df['text_emoji'].tolist())]
senti_df.head()

,sentence,text_only,emo_only,text_emoji,text_senti,emoji_senti,txt_emo_senti
0,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！,紅心,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！_紅心,positive,negative,positive
1,為今日難吃的涼麵做個紀錄😤,為今日難吃的涼麵做個紀錄,臉上有蒸汽從鼻子裡冒出來,為今日難吃的涼麵做個紀錄_臉上有蒸汽從鼻子裡冒出來,negative,positive,negative
2,前幾天的照片真的慘不忍睹😂,前幾天的照片真的慘不忍睹,滿臉喜悅的淚水,前幾天的照片真的慘不忍睹_滿臉喜悅的淚水,negative,negative,negative
3,🌟天婦羅（時蔬）,天婦羅（時蔬）,發光的星星,天婦羅（時蔬）_發光的星星,negative,positive,negative
4,看的真的是又愛又虐😍,看的真的是又愛又虐,心眼笑臉,看的真的是又愛又虐_心眼笑臉,negative,positive,negative


In [12]:
sentis = senti_df[['text_senti',	'emoji_senti',	'txt_emo_senti']]

In [13]:
display(pd.crosstab(sentis.text_senti, sentis.emoji_senti))
display(pd.crosstab(sentis.emoji_senti, sentis.txt_emo_senti))
display(pd.crosstab(sentis.text_senti, sentis.txt_emo_senti))

emoji_senti,negative,positive
text_senti,,
negative,108,150
positive,227,514


txt_emo_senti,negative,positive
emoji_senti,,
negative,180,155
positive,107,557


txt_emo_senti,negative,positive
text_senti,,
negative,194,64
positive,93,648


# 第三部分：衝突 emoji 詞頻

請你找出 `emoji_senti` 和 `txt_emo_senti` 兩個欄位被標成不同情緒的 text 們，並計算出現在這些 text 中的 emoji 的頻率。

**最後列出頻率最高的 10 個 emoji**

🔔 提示 🔔 會用到的欄位: emo_only, emoji_senti, txt_emo_senti

```
⚠️ 注意：若同個 text 中有多個 emoji，請將這些 emoji 分次都算進去⚠️ 

舉例來說， 
有一個 row 是 "推坑大家🐹🐹"
他的 [emoji_senti] 是 positive、[txt_emo_senti] 是 positive
所以這一 row 就是我們符合我們要求的 text。

由於他的 [emo_only] 欄位是 "倉鼠,倉鼠"
計算頻率時 “倉鼠” 要被算到兩次。(而不是一次)
```

> 配分方式：20% (每一個emoji2%)

In [ ]:
'''1. 找出所有emoji_senti 和 txt_emo_senti 兩個欄位情緒不一致的句子中，所有出現的 emoji '''
emo_counts = []
for idx, row in senti_df.iterrows():
  if row['emoji_senti'] != row['txt_emo_senti']:
      emo_counts.append(row['emo_only'])

'''2. 把有多個emoji的list攤開來，分次計算進去'''
emo_counts_flat = []
for e in emo_counts:
  if re.findall(',', e): 
    emo_counts_flat+=e.split(', ') # 前面是以半形逗號＋空白做分割
  else:
    emo_counts_flat.append(e)

In [ ]:
Counter(emo_counts_flat).most_common(10)

[('滿臉喜悅的淚水', 36),
 ('紅心', 26),
 ('懇求的臉', 15),
 ('笑得在地上打滾', 13),
 ('含淚的笑臉', 13),
 ('汗流浹背的笑臉', 10),
 ('大聲哭泣的臉', 6),
 ('向下箭頭', 6),
 ('手摀臉', 6),
 ('心眼笑臉', 5)]

# BONUS (10分）

試著提出一個可行的pipeline（不需寫出真實code 但也不要是空泛的概念）

這個 pipeline要抓出「以emoji造成諷刺效果」的句子，

也就是句子要保留emoji、必須辨識出哪些句子存在諷刺

(e.g., "我覺得他們做得很好欸🤗", "他最聰明了🙄"）

> 也可以使用 chatGPT，說不定他能提供有用的意見喔😇😇😇


本次作業已經使用fine-tuned在中文對話資料上的 RoBERTa （A Robustly Optimized BERT Pretraining Approach)，但結果有時還是不甚理想。

同學在加分題的回答大概分成這幾類：

1. Supervised learning method ( [鮑鈺文](https://github.com/lope-classroom/HW5_emoji_sentiment_analysis-vppyw/blob/main/HW5_emoji_sent_analysis_%E9%AE%91%E9%88%BA%E6%96%87.ipynb))

2. Follow hw's（ [Deborah](https://github.com/lope-classroom/HW5_emoji_sentiment_analysis-deborahwatty/blob/main/HW5_emoji_deborahwatty.ipynb)）

3. Prompt LLM 做emoji翻譯或諷刺偵測( [陳可邦](https://github.com/lope-classroom/HW5_emoji_sentiment_analysis-ChenKB91/blob/main/HW5_emoji_sent_analysis_%E9%99%B3%E5%8F%AF%E9%82%A6.ipynb)、[黃羿文](https://github.com/lope-classroom/HW5_emoji_sentiment_analysis-hachiyee/blob/main/HW5_emoji_sent_analysis_%E9%BB%83%E7%BE%BF%E6%96%87.ipynb)）